In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
BaseUrl = 'https://www.immobiliare.it/vendita-case/trieste-provincia/?criterio=rilevanza'


Allinfo = []

def unify_bagni_bagno(label):
    # Map variations of "bagni" and "bagno" to "Bagni"
    variations = ['bagni', 'bagno']
    if label.lower() in variations:
        return 'Bagni'
    return label

def unify_locali_locale(label):
    # Map variations of "locali" and "locale" to "Locali"
    variations = ['locali', 'locale']
    if label.lower() in variations:
        return 'Locali'
    return label

# Function to get the House' data, one by one and then store each one of them in the "Allinfo" variable
def GetData(Url):
    url = Url
    while True:
        Req = requests.get(url).text
        Soup = BeautifulSoup(Req, 'html.parser')
        HouseList = Soup.find_all('div', class_='nd-mediaObject__content in-card__content in-realEstateListCard__content')
        #PageBtns = Soup.find('a', class_='in-pagination__item')
        div_tag = Soup.find_all('div', class_='in-pagination__control')
        
        NextPageBtn = None
        
        for i in div_tag:
            if i['data-cy'] == 'pagination-next':
                NextPageBtn = i.find('a')
                break
            else:
                pass
            
        

        for house in HouseList:
            title = house.find('a', class_='in-card__title').text
            house_url = house.find('a')['href']

            price_element = house.find('li', class_='nd-list__item in-feat__item in-feat__item--main in-realEstateListCard__features--main')
            price = price_element.text.strip() if price_element else 'N/A'

            HouseListOutput = {
                'Title': title,
                'Url': house_url,
                'Price': price
            }
            
            elements = house.find_all('li', class_='nd-list__item in-feat__item')
            for element in elements:
                label = element.get('aria-label')
                value = element.find('div', class_='in-feat__data').text.strip()

                if label == 'superficie':
                    unit = element.find('span', class_='in-mainFeatures__text').text
                    value = value.replace('m²', '')  # Remove existing 'm²' if present
                    value += unit
                    
                label = unify_bagni_bagno(label)  # Unify variations of "bagni" and "bagno"
                label = unify_locali_locale(label)  # Unify variations of "locali" and "locale"
                HouseListOutput[label] = value
            
            
            Allinfo.append(HouseListOutput)
            print(HouseListOutput)
            #print(NextPageBtn)
            
            
        if NextPageBtn:
            url = NextPageBtn['href']         
        else:
            break
            
        

GetData(BaseUrl)
Allinfo

{'Title': 'Appartamento via Giuseppe Picciola 3, San Vito - Campi Elisi, Trieste', 'Url': 'https://www.immobiliare.it/annunci/104175763/', 'Price': '€ 430.000', 'Locali': '5', 'superficie': '122m²', 'Bagni': '2'}
{'Title': 'Villa unifamiliare Strada Costiera 70, Costiera - Grignano, Trieste', 'Url': 'https://www.immobiliare.it/annunci/104339741/', 'Price': '€ 1.650.000', 'Locali': '5+', 'superficie': '400m²', 'Bagni': '3+', 'lusso': ''}
{'Title': 'Quadrilocale via Pasquale Besenghi 6, San Giacomo, Trieste', 'Url': 'https://www.immobiliare.it/annunci/101944724/', 'Price': '€ 385.000', 'Locali': '4', 'superficie': '136m²', 'Bagni': '1'}
{'Title': 'Bilocale Strada di Portopiccolo, Duino-Aurisina', 'Url': 'https://www.immobiliare.it/annunci/98582788/', 'Price': '€ 497.000', 'Locali': '2', 'superficie': '92m²', 'Bagni': '1', 'piano': '4'}
{'Title': 'Appartamenti e Attici di nuova costruzione a Trieste', 'Url': 'https://www.immobiliare.it/annunci/99036760/', 'Price': 'da € 160.000', 'Locali'

[{'Title': 'Appartamento via Giuseppe Picciola 3, San Vito - Campi Elisi, Trieste',
  'Url': 'https://www.immobiliare.it/annunci/104175763/',
  'Price': '€ 430.000',
  'Locali': '5',
  'superficie': '122m²',
  'Bagni': '2'},
 {'Title': 'Villa unifamiliare Strada Costiera 70, Costiera - Grignano, Trieste',
  'Url': 'https://www.immobiliare.it/annunci/104339741/',
  'Price': '€ 1.650.000',
  'Locali': '5+',
  'superficie': '400m²',
  'Bagni': '3+',
  'lusso': ''},
 {'Title': 'Quadrilocale via Pasquale Besenghi 6, San Giacomo, Trieste',
  'Url': 'https://www.immobiliare.it/annunci/101944724/',
  'Price': '€ 385.000',
  'Locali': '4',
  'superficie': '136m²',
  'Bagni': '1'},
 {'Title': 'Bilocale Strada di Portopiccolo, Duino-Aurisina',
  'Url': 'https://www.immobiliare.it/annunci/98582788/',
  'Price': '€ 497.000',
  'Locali': '2',
  'superficie': '92m²',
  'Bagni': '1',
  'piano': '4'},
 {'Title': 'Appartamenti e Attici di nuova costruzione a Trieste',
  'Url': 'https://www.immobiliare.i

In [3]:
df = pd.DataFrame(Allinfo)

df

,Title,Url,Price,Locali,superficie,Bagni,lusso,piano,tipologie,data vendita
0,"Appartamento via Giuseppe Picciola 3, San Vito...",https://www.immobiliare.it/annunci/104175763/,€ 430.000,5,122m²,2,NaN,NaN,NaN,NaN
1,"Villa unifamiliare Strada Costiera 70, Costier...",https://www.immobiliare.it/annunci/104339741/,€ 1.650.000,5+,400m²,3+,,NaN,NaN,NaN
2,"Quadrilocale via Pasquale Besenghi 6, San Giac...",https://www.immobiliare.it/annunci/101944724/,€ 385.000,4,136m²,1,NaN,NaN,NaN,NaN
3,"Bilocale Strada di Portopiccolo, Duino-Aurisina",https://www.immobiliare.it/annunci/98582788/,€ 497.000,2,92m²,1,NaN,4,NaN,NaN
4,Appartamenti e Attici di nuova costruzione a T...,https://www.immobiliare.it/annunci/99036760/,da € 160.000,1 - 4,57m²,NaN,NaN,NaN,10,NaN
...,...,...,...,...,...,...,...,...,...,...
1467,"Palazzo - Edificio piazza dell'Unità d'Italia,...",https://www.immobiliare.it/annunci/90640867/,€ 1.000.000,NaN,1m²,NaN,NaN,60,NaN,NaN
1468,"Villa unifamiliare Località Basovizza 218, Bas...",https://www.immobiliare.it/annunci/103293662/,€ 400.000,NaN,800m²,NaN,NaN,NaN,NaN,NaN
1469,"Trilocale via Bartolomeo d'Alviano 68, Chiarbo...",https://www.immobiliare.it/annunci/104591885/,€ 129.000,3,75m²,1,NaN,2,NaN,NaN
1470,Terratetto unifamiliare frazione San Pelagio 4...,https://www.immobiliare.it/annunci/104662199/,€ 345.000,5,147m²,2,NaN,NaN,NaN,NaN


### Function to get some individual House details

In [4]:
def GetIndividualData(Url):
    url = Url
    Req = requests.get(url).text
    Soup = BeautifulSoup(Req, 'html.parser')
    
    div_element = Soup.find('div', class_='in-readAll')
    
    if div_element:
        # Find the inner <div> element and extract its text content
        inner_div = div_element.find('div')
        data = inner_div.text.strip()
        return data
    else:
        return None


In [5]:
GetIndividualData('https://www.immobiliare.it/annunci/104339741/')

"Splendida villa di ampia metratura in posizione unica sulla strada costiera, sviluppata su tre livelli è completamente circondata dal suo parco (circa 2.000 metri) sovrastante uno dei più meravigliosi tratti di costa triestina. Dal suo giardino godiamo di una vista mare stupenda, lo stesso mare che possiamo raggiungere attraverso un accesso privato dove troviamo il molo in concessione demaniale esclusiva. L'immobile è così distribuito: al piano terra (ca 150mq) troviamo due stanze, un ampia sala con caminetto e vista sul mare e il giardino frontale, una cucina abitabile, un bagno e un wc;al primo piano (ca 150 mq) troviamo la zona notte, con 4 stanze, di cui una con un pittoresco poggiolo con vista mozzafiato, e 2 bagni completi;La proprietà si completa di una splendida taverna (ca 80 mq), dotata di caminetto e cucina, con diretto accesso al giardino privato, dove troviamo una sauna per due persone, un'ampia doccia con vista e un bagno. La proprietà ha accesso privato al mare dove in 

### Getting an individual detail like Descrizione and then appending as an extra column called Deecrizione

In [6]:
df['Descrizione'] = df['Url'].apply(GetIndividualData)      # First time I see the apply function for a dataframe, I love that bro!!
df

In [ ]:
df

,Title,Url,Price,Locali,superficie,Bagni,lusso,piano,tipologie,data vendita,Descrizione
0,"Appartamento via Giuseppe Picciola 3, San Vito...",https://www.immobiliare.it/annunci/104175763/,€ 430.000,5,122m²,2,NaN,NaN,NaN,NaN,"VERDEMARE, nell'edificio più ""alto"" (Via Picci..."
1,"Villa unifamiliare Strada Costiera 70, Costier...",https://www.immobiliare.it/annunci/104339741/,€ 1.650.000,5+,400m²,3+,,NaN,NaN,NaN,Splendida villa di ampia metratura in posizion...
2,"Bilocale Strada di Portopiccolo, Duino-Aurisina",https://www.immobiliare.it/annunci/98582788/,€ 497.000,2,92m²,1,NaN,4,NaN,NaN,PRIMO INGRESSO - Nell’esclusivo complesso di P...
3,Appartamenti e Attici di nuova costruzione a T...,https://www.immobiliare.it/annunci/99036760/,da € 160.000,1 - 4,57m²,NaN,NaN,NaN,10,NaN,Le Terrazze sul Golfo è il nuovo progetto resi...
4,"Attico viale Miramare 109, Barcola, Trieste",https://www.immobiliare.it/annunci/85478590/,€ 650.000,3,105m²,2,,NaN,NaN,NaN,Meraviglioso attico nel residence nel compless...
...,...,...,...,...,...,...,...,...,...,...,...
1463,"Appartamento piazza del Sansovino 2/1, San Gia...",https://www.immobiliare.it/annunci/102984936/,€ 125.000,NaN,70m²,NaN,NaN,6,NaN,NaN,"appartamento anni 70, sesto piano, in buone co..."
1464,"Palazzo - Edificio piazza dell'Unità d'Italia,...",https://www.immobiliare.it/annunci/90640867/,€ 1.000.000,NaN,1m²,NaN,NaN,60,NaN,NaN,"Ricerco, per pronto acquisto, immobile terra-c..."
1465,"Villa unifamiliare Località Basovizza 218, Bas...",https://www.immobiliare.it/annunci/103293662/,€ 400.000,NaN,800m²,NaN,NaN,NaN,NaN,NaN,Casa composta da cantina piano fuori terra e g...
1466,"Quadrilocale via della Tesa 18, Settefontane, ...",https://www.immobiliare.it/annunci/104591655/,€ 229.000,4,120m²,2,NaN,1,NaN,NaN,"Proponiamo in zona molto servita, a pochi minu..."


### Gettting individual Urls

In [ ]:
def get_values_for_key(dictionaries, key):
    values = []
    for dictionary in dictionaries:
        if key in dictionary:
            values.append(dictionary[key])
    return values

In [ ]:
urls = get_values_for_key(Allinfo, 'Url')
print(urls)

['https://www.immobiliare.it/annunci/104175763/', 'https://www.immobiliare.it/annunci/104339741/', 'https://www.immobiliare.it/annunci/98582788/', 'https://www.immobiliare.it/annunci/99036760/', 'https://www.immobiliare.it/annunci/85478590/', 'https://www.immobiliare.it/annunci/97341708/', 'https://www.immobiliare.it/annunci/99047388/', 'https://www.immobiliare.it/annunci/102963330/', 'https://www.immobiliare.it/annunci/104470555/', 'https://www.immobiliare.it/annunci/103223490/', 'https://www.immobiliare.it/annunci/101723507/', 'https://www.immobiliare.it/annunci/101644883/', 'https://www.immobiliare.it/annunci/103828865/', 'https://www.immobiliare.it/annunci/103449562/', 'https://www.immobiliare.it/annunci/103618852/', 'https://www.immobiliare.it/annunci/104561329/', 'https://www.immobiliare.it/annunci/87393246/', 'https://www.immobiliare.it/annunci/101056447/', 'https://www.immobiliare.it/annunci/101056441/', 'https://www.immobiliare.it/annunci/100559370/', 'https://www.immobiliare.

In [ ]:
import concurrent.futures
import requests
from bs4 import BeautifulSoup
import pandas as pd

def GetIndividualData(url):
    session = requests.Session()
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    div_element = soup.find('div', class_='in-readAll')

    if div_element:
        inner_div = div_element.find('div')
        data = inner_div.text.strip()
        return data
    else:
        return None

def process_url(url):
    data = GetIndividualData(url)
    return data

# Assuming df is your Pandas dataframe with 'Url' column
urls = df['Url'].tolist()

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_url, urls)

df['Descrizione'] = list(results)


In [ ]:
df

,Title,Url,Price,Locali,superficie,Bagni,lusso,piano,tipologie,data vendita,Descrizione
0,"Appartamento via Giuseppe Picciola 3, San Vito...",https://www.immobiliare.it/annunci/104175763/,€ 430.000,5,122m²,2,NaN,NaN,NaN,NaN,"VERDEMARE, nell'edificio più ""alto"" (Via Picci..."
1,"Villa unifamiliare Strada Costiera 70, Costier...",https://www.immobiliare.it/annunci/104339741/,€ 1.650.000,5+,400m²,3+,,NaN,NaN,NaN,Splendida villa di ampia metratura in posizion...
2,"Bilocale Strada di Portopiccolo, Duino-Aurisina",https://www.immobiliare.it/annunci/98582788/,€ 497.000,2,92m²,1,NaN,4,NaN,NaN,PRIMO INGRESSO - Nell’esclusivo complesso di P...
3,Appartamenti e Attici di nuova costruzione a T...,https://www.immobiliare.it/annunci/99036760/,da € 160.000,1 - 4,57m²,NaN,NaN,NaN,10,NaN,Le Terrazze sul Golfo è il nuovo progetto resi...
4,"Attico viale Miramare 109, Barcola, Trieste",https://www.immobiliare.it/annunci/85478590/,€ 650.000,3,105m²,2,,NaN,NaN,NaN,Meraviglioso attico nel residence nel compless...
...,...,...,...,...,...,...,...,...,...,...,...
1463,"Appartamento piazza del Sansovino 2/1, San Gia...",https://www.immobiliare.it/annunci/102984936/,€ 125.000,NaN,70m²,NaN,NaN,6,NaN,NaN,"appartamento anni 70, sesto piano, in buone co..."
1464,"Palazzo - Edificio piazza dell'Unità d'Italia,...",https://www.immobiliare.it/annunci/90640867/,€ 1.000.000,NaN,1m²,NaN,NaN,60,NaN,NaN,"Ricerco, per pronto acquisto, immobile terra-c..."
1465,"Villa unifamiliare Località Basovizza 218, Bas...",https://www.immobiliare.it/annunci/103293662/,€ 400.000,NaN,800m²,NaN,NaN,NaN,NaN,NaN,Casa composta da cantina piano fuori terra e g...
1466,"Quadrilocale via della Tesa 18, Settefontane, ...",https://www.immobiliare.it/annunci/104591655/,€ 229.000,4,120m²,2,NaN,1,NaN,NaN,"Proponiamo in zona molto servita, a pochi minu..."


## My honest thoughts on the sample script:

I must say, the idea was to give a sample... This script is amazing!!

The way you write code cleanly, use libraries, logic, and even the way you name variables, it's everything very professional!!

Besides that, this shows how accountable you are with with the projects you are working on and that you're an incredibly professional and trustworthy person.

I'm so glad I found an amazing developer for this project to work by my side, you're gonna do amazingly well, congrats!

## Suggestions

- I figure that, since there's a need to get inside each page to get as much data as possible, we could just get the properties urls from the base URL, paginate through all the pages and get the url for each property, then scrape them all individually with another function, no need to scrape information from the properties on the base URL.
- I saw on the first code block that you got the next page url from the pagination button. To save time we could just get the total number of properties (**1.491** risultati per:case in vendita in provincia di Trieste) and divide it by the total number of properties per page, round the number up and that would be the amount of pages, then you just make a loop changing the page parameter: https://www.immobiliare.it/vendita-case/trieste-provincia/?criterio=rilevanza&pag={i} (i being the number of the page).
- The function to get into each property's url is quite resource intensive (it's been 20 minutes already), therefore, I was planning to use it to get a database of properties, get them all, then, only scrape the new ones deeply, but that's for the future of the project, last stages.

## Next steps

Now the next steps are:

- Seeing if the other 3 websites are scrapable by bs4.
- finding where the requested data by the client is located at each website.
- start prototyping.